In [96]:
"""This script is to update workqueue table in SQL for KPI Dashboard"""

#Importing nessary libraries
import os
import datetime
import shutil
import sys
import pandas as pd
import time
import numpy as np
import calendar
from calendar import monthrange
import warnings
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")
from importlib import reload
import pyodbc
import pandas as pd

reload(sys)
pd.options.mode.chained_assignment = None


start = time.time()

now = datetime.datetime.now()

print(f"The process started at {now}")


#This is the main folderpath
folder = r'\\filesrvwhq\PowerWorks_Ops\Ambulatory Services\Client and Team Folders\0_New_Client_Folders'

#getting the right month folder
month_input = input("Please input month: ")
month_namefin = month_input.title()
month_name = month_input[0:3]
datetime_object = datetime.datetime.strptime(month_name, "%b")
month_number = datetime_object.month
current_year = now.year
first_date="01"
if month_number > 9:
    month_number = str(month_number)
    
else:
    month_number = "0"+str(month_number)
month_date = (str(current_year)+"-"+month_number+"-"+first_date)
real_date = datetime.datetime.strptime(month_date, "%Y-%m-%d")

if real_date.weekday() < 5:
    x = 5 - real_date.weekday()
    x2 = int(first_date) + x
    
if real_date.weekday() == 5:
    x2 = 1
    
else:
    x = real_date.weekday() - 5
    x2 = int(first_date) - x
    
week_staurday_list = list(range(x2,32,7))  
week_folder_list = []
for i in week_staurday_list:
    if i > 9:
        week_folder_list.append(month_number+"_"+str(i)+"_"+str(current_year))
    else:
        week_folder_list.append(month_number+"_"+"0"+str(i)+"_"+str(current_year))
week_folder_list.append("EOM")
        
        
        
print(f"The suggestion for folders are \n{week_folder_list}")
week_name = input("Please paste one folder name from above: ")
current_year = str(2020)
Last_day = monthrange(int(current_year), int(month_number))
count_of_week = len(calendar.monthcalendar(int(current_year),int(month_number)))
c = calendar.Calendar(firstweekday=calendar.SUNDAY)
year = int(current_year); month = int(month_number)
monthcal = c.monthdatescalendar(year,month)
First_Saturday_Date = [day for week in monthcal for day in week if day.weekday() == calendar.SATURDAY and day.month == month][0]
year, month, First_Saturday = str(First_Saturday_Date).split('-')


#Getting the list of client folders
list_of_folder = os.listdir(path=folder)

#Creating folder with path list
folderin_list = []
for i in list_of_folder:
    folderin = os.path.join(folder,i)
    folderin_list.append(folderin)
    
client_list = ['Barnabas_BARN_HS', 'Escambia_ECHA_AL','Humboldt_HUMB_NV']
    
#Getting the desired client folders

client_folder_list = []
for i in folderin_list:
    for j in client_list:
        if j in i:
            client_folder_list.append(i+"\\"+current_year+"\\"+month_number+"_"+str(month_input.title())+"\\"+week_name)

            
#Start and end date of the week
start_day=1
end_day=int(Last_day[1])            
            
#fucntion to remove pseudo files

def remove_pseudo(x):
    for i in x:
        size = os.path.getsize(i)
        if size < 2000:
            x.remove(i)
    return x


#Getting the desired list of files
if week_name != "EOM":
    file_name = 'EATB Weekly'
else:
    file_name = 'EATB EOM'
eatb_list = []
for i in client_folder_list:
        file_list = os.listdir(i)
        for j in file_list:
            if file_name in j:
                eatb_list.append(i+'\\'+j)

eatb = pd.DataFrame()

for i in eatb_list:
    df = pd.read_excel(i, sheet_name = "Enhanced_ATB")
    df["Client"] = i.split("\\")[7]
    eatb = eatb.append(df, ignore_index=True)
    
#defining a function for getting right column names


#EATB

#defining a function for getting right column names


def column_check (column1,column2,data):
    if column1 in data.columns:
        return column1
    else:
        return column2

facility_EATB = column_check('Patient Facility', 'Facility',eatb)

Billing_Entity_EATB = column_check('Billing Entity Name', 'Billing Entity',eatb)


if 'Current Financial Class' in eatb.columns:
    atb_fin='Current Financial Class'
else:
    atb_fin= 'Responsible Financial Class'

if 'Current Health Plan' in eatb.columns:
    atb_plan='Current Health Plan'
else:
    atb_plan= 'Responsible Health Plan'



if 'Encounter Balance' in eatb.columns:
    Balance='Encounter Balance' 
else:
    Balance= 'Balance Amount'

#Converting the NaT pandas values to None type
def remove_NaT(df):
    for i in df:
        for n,j in enumerate(i):
            if j is pd.NaT:
                i[n] = None

remove_NaT(eatb)

eatb = eatb[[Billing_Entity_EATB, facility_EATB, 'Discharge Aging Category','Discharge Aging Range',atb_fin,atb_plan,
             'Insurance Balance', 'Balance Type', 'Balance Status', Balance, 'Billed Status','DNFB Status', 'Medical Service',
             'Discharge Date','Activity Date','Self-Pay Balance','Unbilled Aging Category', 'Encounter Number',
             'Encounter Last Touch Date',
             'Client' ]]

remove_NaT(eatb)

eatb.reset_index(drop=True)
eatb = eatb.to_numpy()
print("EATB Uploaded")


""" Starting the SQL update using pyodbc library. The values in the tables are converted to numpy arrays so that the same can be appended using for loop into desired SQL Server"""


#Connecting to SQL Server
server_name = "W1751904\LOCAL_CERNER"
database_name = "KPI Dashboard"
conn = pyodbc.connect(Driver='{SQL Server Native Client 11.0}',
                      Server=server_name,
                      Database=database_name,
                      trusted_connection='yes')

 


for i in eatb:
    if(type(i[11])==float):
        i[11]="NA"  

insert_eatb = """INSERT INTO EATB ( [Billing Entity], [Facility],[Discharge Aging Category],[Discharge Aging Range],
                    [Responsible Financial Class], [Responsible Health Plan],
             [Insurance Balance], [Balance Type], 
             [Balance Status], [Balance Amount], [Billed Status], 
             [DNFB Status], [Medical Service], [Discharge Date],[Activity Date],
             [Self-Pay Balance],[Unbilled Aging Category],[Encounter Number],[Encounter Last Touch Date],[Client]) 
             VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?) """
 
remove_NaT(eatb)

 

for row in eatb:
    

    values_eatb = (str(row[0]),str(row[1]),str(row[2]),str(row[3]),str(row[4]), str(row[5]),row[6],str(row[7]),str(row[8]),row[9],str(row[10]),row[11],str(row[12]),row[13],row[14],row[15],
                  str(row[16]),str(row[17]),row[18],str(row[19]))
    conn.execute(insert_eatb,values_eatb)
                           


conn.commit()
print("EATB is uploaded to SQL Server")  

end = time.time()

print(f"The task was completed in {(end-start)/60} minutes")

The process started at 2020-11-30 19:33:34.542710
Please input month: October
The suggestion for folders are 
['10_03_2020', '10_10_2020', '10_17_2020', '10_24_2020', '10_31_2020', 'EOM']
Please paste one folder name from above: 10_03_2020
WorkQueue Uploaded
WorkQueue - Uploaded to SQL
The task was completed in 115.42037642002106 minutes


In [94]:
workqueue_list

['Work Queue Weekly - WE 10.03.20 - Barnabas.xlsx',
 'Work Queue Weekly - WE 10.03.20 - ECHA_AL.xlsx',
 'Work Queue Weekly - WE 10.03.20 - HUMB_NV.xlsx']

array([['Benewah Community Hospital',
        'Alexander (RevVWx) BENEID, Vijay Charles', 'Client', ...,
        'AARP HEALTHCARE OPTION', 'OP', 'Benewah'],
       ['Benewah Community Hospital',
        'Alexander (RevVWx) BENEID, Vijay Charles', 'Client', ...,
        'MISC Commercial', 'Swingbed', 'Benewah'],
       ['Benewah Community Hospital',
        'Alexander (RevVWx) BENEID, Vijay Charles', 'Client', ...,
        'MISC Commercial', 'Swingbed', 'Benewah'],
       ...,
       ['Benewah Community Hospital',
        'Alexander (RevVWx) BENEID, Vijay Charles', 'Client', ...,
        'REGENCE BLUESHIELD OF IDAHO', 'Recurring Outpatient', 'Benewah'],
       ['Benewah Community Hospital', 'Minier,Lori', 'Client', ..., 0,
        'Reference Lab', 'Benewah'],
       ['Benewah Community Hospital', 'Thomas (RevWx) BENEID, Sanjana',
        'Client', ..., 'MEDICAID OF IDAHO', 'Recurring Outpatient',
        'Benewah']], dtype=object)

WorkQueue - Uploaded to SQL


0 Benewah Community Hospital
1 Minier,Lori
2 Client
3 COVID 19 Statement Hold
4 187789
5 100.0
6 2020-08-18 00:00:00
7 2020-08-18 00:00:00
8 31-60
9 Benewah Hosp
10 2020-10-04 00:00:00
11 47
12 31+ Days
13 31-60 Days
14 Debit
15 nan
16 None
17 None
18 Non Patient
19 Benewah


In [77]:
for i in workqueue[2]:
    print(type(i))

<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'float'>
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'str'>
<class 'str'>
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'int'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'float'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>


In [78]:
workqueue[3]

array(['St. Maries Family Medicine',
       'Alexander (RevVWx) BENEID, Vijay Charles', 'Client',
       'Pending Reimbursement Claim', '192451', 156.01,
       Timestamp('2020-10-01 00:00:00'), Timestamp('2020-10-01 00:00:00'),
       '0-30', 'St. Maries Family Medicine',
       Timestamp('2020-10-04 00:00:00'), 3, 'Less than 72 hours',
       '0-7 Days', 'Debit', 156.01, 'Medicaid',
       'MEDICAID FOR RURAL HEALTH', 'Clinic', 'Benewah'], dtype=object)

In [1]:
import numpy as np

In [12]:
new = np.array([[23,23],[24,24,25]])

In [13]:
for i in new:
    print(i)


[23, 23]
[24, 24, 25]
